In [1]:
from data_tools import parse_boxes_from_json, get_filenames
from tiling import crop_ground_truths, get_random_tile
from targets import generate as gen
from visualize import render_boxes
from shapely.geometry import Polygon
import numpy as np
import matplotlib.pyplot as plt
from tkinter.filedialog import askopenfilename
import cv2
from pipeline_v2 import get_dataset, _get_filenames, _generate_tiles

%matplotlib inline

In [2]:
txt_path = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/txt/D0117-5755036.txt'
json_path = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/json/D0117-5755036.json'
img_path = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/images/D0117-5755036.tiff'

In [3]:
img_dir = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/images'
json_dir = 'D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD/data/general_dataset/json'

In [4]:
ds = get_dataset(img_dir, json_dir, '*')

Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [5]:

ds

<_MapAndBatchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>

In [6]:
ds = ds.take(1)
e = list(ds.as_numpy_iterator())

In [ ]:
ds = ds.take(10)
a = ds.as_numpy_iterator()

In [ ]:
args = lambda x : (x[0].astype(int), np.transpose(x[1], (2, 0, 1)))

In [ ]:
render_boxes(*args(list(next(a))))

In [15]:
e[0][3].shape

(16, 128, 128, 1)

In [ ]:
points, polygons, _ = parse_boxes_from_json(json_path)
img = plt.imread(img_path)

In [ ]:
img_files, gt_files = _get_filenames(img_dir, json_dir, '*', 'tiff', 'json')

In [ ]:
tilegen = _generate_tiles(512, img_files, gt_files)

In [ ]:
#plt.imshow(next(tilegen)[0])
#plt.show()
print(np.transpose(next(tilegen)[1], (2, 0, 1)).shape)

In [ ]:
image, ground_truth = next(tilegen)
render_boxes(image, np.transpose(ground_truth, (2, 0, 1)))

In [ ]:
img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
k = cv2.fillPoly(img, pts=np.transpose(points.astype(np.int32), (2, 0, 1)), color=(0, 0, 0)) # NOTE: check whether the points are in the shape (x, y, boxes) or (boxes, x, y)
resized_img = cv2.resize(img, (1280, 720)) # NOTE: Resize image to fit in the screen. Otherwise, I must find how to enable the cv2.window_normal option

In [ ]:
cv2.imshow('a name', resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()